In [ ]:
# default_exp core

# MuModel

> Establish a muzero model using pytorch

In [ ]:
# exports 

from nbdev.showdoc import *
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import namedtuple
from typing import Tuple, List, Union

def set_seed(seed: int=47) -> None:
    torch.manual_seed(seed)
    np.random.seed(seed)
    
# Stacks the columns on top of each other. Union lets us define two types cool!
def bstack(bb: List[Union[float, np.ndarray]]) -> List[np.ndarray]:
    return [np.array([i[j] for i in bb]).reshape(len(bb), -1) for j in range(len(bb[0]))]

def to_one_hot(a: np.ndarray, K: int, a_dim: int) -> np.ndarray:
    # Vectorized version of one hot encoding.
    # K is number of steps and a_dim is the action dimension.
    one_hot_action = np.zeros((K * a_dim))
    index = np.arange(0, K * a_dim, a_dim)
    # Removes negative actions
    index = index[a >= 0]
    one_hot_action[a[a >= 0] + index] = 1
    return np.split(one_hot_action, K)

def reformat_batch(batch: np.ndarray, K: int, a_dim: int, remove_policy=False) -> Tuple[List[np.ndarray]]:
    X, Y = [], []
    for o, a, outs in batch:
        a = np.array(a)
        x = [o] + to_one_hot(a, K, a_dim)
        
        # Flatten outs into a list for targets.
        y = [item for sublist in outs for item in sublist]
        
        X.append(x)
        Y.append(y)
        
        X, Y = bstack(X), bstack(Y)
        
        if remove_policy:
            nY = [Y[0]]
            # The model predicts three future quantities.
            for i in range(3, len(Y), 3):
                nY.append(Y[i])
                nY.append(Y[i+1])
            Y = nY
        else:
            # Removes the reward prediction??
            Y.pop(1)
        
        return X, Y
    

class DenseRepresentation(nn.Module):
    # h network: Gets the representation for the initial hidden state s^0 from past observations.
    def __init__(self, o_dim: int, s_dim: int, hidden_layer: int, hidden_layer_count: int) -> None:
        super().__init__()
        
        # Input linear layer with elu activation function.
        sequential = [nn.Linear(o_dim, hidden_layer_dim), nn.ELU()]
        # Hidden llinear layers with elu activation function.
        sequential += [nn.Linear(hidden_layer_dim, hidden_layer_dim), nn.ELU()] * hidden_layer_count
        # Output linear layer according to state dimension.
        self.out_state = nn.Linear(hidden_layer_dim, s_dim)
        
        # Star notation makes the function take in a tuple :)
        self.linearReluStack = nn.Sequential(*tuple(sequential))
    
    def forward(self, state: torch.Tensor) -> torch.Tensor:
        x = self.linearReluStack(state)
        return self.out_state(x)

class DenseDynamic(nn.Module):
    # g network: Produces an immediate reward r and and a new hidden state s.
    def __init__(self, s_dim: int, a_dim: int, hidden_layer_dim: int, hidden_layer_count: int) -> None:
        super().__init__()
        
        # Input and hidden linear layers with elu activation function.
        sequential = [nn.Linear(s_dim + a_dim, hidden_layer_dim), nn.ELU()]
        sequential += [nn.Linear(hidden_layer_dim, hidden_layer_dim), nn.ELU()] * hidden_layer_count
        
        self.linearReluStack = nn.Sequential(*tuple(sequential))
        self.out_reward = nn.Linear(hidden_layer_dim, 1)
        self.out_state = nn.Linear(hidden_layer_dim, s_dim)
    
    def forward(self, state: torch.Tensor, action: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Why are the dimensions reversed?? Takes the latest state and action first??? Then reverse it again. 
        x = self.linearRelueStack(state.T, action.T).T
        return self.out_reward(x), self.out_state(x)
    
class DensePrediction(nn.Module):
    # f network: Computes the policy p^k and and value function v^k from the hidden state s^k.
    def __init__(self, s_dim: int, a_dim: int, hidden_layer_dim: int, hidden_layer_count: int, 
                 with_policy: bool = True) -> None:
        super().__init__()
        self.with_policy = with_policy
        
        sequential = [nn.Linear(s_dim, hidden_layer_dim), nn.ELU()]
        sequential += [nn.Linear(hidden_layer_dim, hidden_layer_dim), nn.ELU()] * hidden_layer_count
        
        self.linearReluStack = nn.Sequential(*tuple(sequential))
        self.out_policy = nn.Linear(hidden_layer_dim, a_dim)
        self.out_value = nn.Linear(hidden_layer_dim, 1)
        
        def forward(self, state: torch.Tensor) -> Union[Tuple[torch.Tensor, torch.Tensor], torch.Tensor]:
            x = self.linearReluStack(state)
            if self.with_policy:
                return self.out_policy(x), self.out_value(x)
            else:
                return self.out_value(x)

class MuModel:
    # these are static variables for this class
    LAYER_COUNT = 4
    LAYER_DIM = 128
    BN = False
    
    def __init__(self, observation_dim: int, action_dim: int, s_dim: int=8, K: int=5, lr: float=1e-3, 
                 with_policy: bool=True, device='cpu') -> None:
        
        self.observation_dim, self.action_dim, self.s_dim = observation_dim, action_dim, s_dim
        self.K, self.lr, self.with_policy = K, lr, with_policy
        self.device = device
        
        # h: Representation function.
        self.h = DenseRepresentation(o_dim=observation_dim[0], s_dim=s_dim, hidden_layer_dim=self.LAYER_DIM, 
                                     hidden_layer_count=self.LAYER_COUNT).to(device)
        # g: Dynamic function.
        self.g = DenseDynamic(s_dim=s_dim, a_dim=action_dim, hidden_layer_dim=self.LAYER_DIM,
                              hidden_layer_count=self.LAYER_COUNT).to(device)
        # f: Predictive function.
        self.f = DensePrediction(s_dim=s_dim, a_dim=action_dim, hidden_layer_dim=self.LAYER_DIM,
                                hidden_layer_count=self.LAYER_COUNT, with_policy=with_policy).to(device)
        
        params = list(self.h.parameters()) + list(self.g.parameters()) + list(self.f.parameters())
        self.optimizer = optim.Adam(params, lr=self.lr)
        self.losses = [] # Overall loss
        
        # Make class compatible with Geohot's other code.
        self.o_dim = self.observation_dim
        self.a_dim = self.action_dim
        Mu = namedtuple('mu', 'predict')
        self.mu = Mu(self.predict)
        
    def forward(self, X: List[torch.Tensor], train: bool=True) -> List[torch.Tensor]:
        # Sets the models into evaluation mode.
        self.h.eval(), self.g.eval(), self.f.eval()
        # Sets the models into training mode.
        if train:
            self.h.train(), self.g.train(), self.f.train()
        
        # Converts features from numpy array to tensor type. 
        X = [torch.from_numpy(x.astype(np.float32)) for x in X]
        Y_pred = []
        
        # Gets the initial hidden state. 
        state = self.h(X[0])
        if self.with_policy:
            policy, value = self.f(state)
            Y_pred += [value, policy]
        else:
            value = self.f(state)
            Y_pred.append(value)
        
        # Make predictions for K time-steps.
        for k in range(self.K):
            reward, new_state = self.g(state, X[k+1])
            if self.with_policy:
                policy, value = self.f(state)
                Y_pred += [value, reward, policy]
            else:
                value = self.f(state)
                Y_pred += [value, reward]
            
            state = new_state
        
        return Y_pred
        
            
    def predict(self, X: List[torch.Tensor]) -> List[torch.Tensor]:
        with torch.no_grad():
            Y_pred = self.forward(X, train=False)
        return Y_pred
    
    def train(self, batch: List[np.ndarray]) -> None:
        # Sets the models into training mode.
        self.h.train(), self.g.train(), self.f.train()
        batch_losses = []
        # Could probably swap F.mse_loss to nn.MSELoss()
        mse, smcel = F.mse_loss, nn.BCEWithLogitsLoss()
        
        X, Y = reformat_batch(batch, self.K, self.action_dim, remove_policy=not self.with_policy)
        Y = [torch.from_numpy(y.astype(np.float32)).to(self.device) for y in Y]
        Y_pred = self.forward(X, train=True)
        
        # Calculates value/policy error from initial hidden state.
        batch_losses.append(mse(Y_pred[0], Y[0]))
        if self.with_policy:
            losses.append(smcel(Y_pred[2], Y[2]))
        
        for k in range(self.K):
            batch_losses.append(mse(Y_pred[3*k + 2]), Y[3*k + 2]) # Value 
            batch_losses.append(mse(Y_pred[3*k + 3]), Y[3*k + 3]) # Reward
            if self.with_policy:
                losses.append(mse(Y_pred[3*k + 4]), Y[3*k + 4]) # Policy
            
        loss = sum(batch_losses)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.losses.append([loss.item()] + [a.item() for a in batch_losses])
    
    def ht(self, state: Union[np.ndarray, torch.Tensor]) -> torch.Tensor:
        with torch.no_grad():
            if not torch.is_tensor(state):
                state = torch.from_numpy(state.astype(np.float32)).to(self.device)
            return self.h(state)
               
    def ft(self, state: Union[np.ndarray, torch.Tensor]) -> torch.Tensor:
        with torch.no_grad():
            if not torch.is_tensor(state):
                state = torch.from_numpy(state.astype(np.float32)).to(self.device)
            
            if self.with_policy:
                policy, value = self.f(state)
                return policy.exp(), value
            else:
                value = self.f(state)
                return value

In [ ]:
bb = np.array([[1, 2, 3], [4, 4, 1]])
bb

array([[1, 2, 3],
       [4, 4, 1]])

In [ ]:
bstack(bb)

[array([[1],
        [4]]),
 array([[2],
        [4]]),
 array([[3],
        [1]])]

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
